# In this notebook the MLABite results are injected in the DB

"""
Ingest "global_evaluation" CSV into the existing BESSER-generated SQLite DB.

What it does (high level):
- Opens DB and introspects table columns with PRAGMA.
- Ensures a Project, Tool, Evaluation, Datashape exist.
- Creates one Dataset per CSV slice (by language if 'language' column exists; else a single 'global' dataset).
- Creates one Observation per slice.
- For every numeric CSV column (except ignore list), ensures a Metric exists and inserts a Measure linked to the Observation.
- Writes values, plus optional unit, uncertainty, error if present/desired.

Customize the CONFIG section as needed.
"""

## Load libs

In [1]:
import sqlite3
import pandas as pd
from datetime import datetime
from pathlib import Path
import json

In [2]:
from datetime import timezone

when_now = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S%z")
print(when_now)

2025-10-30 12:45:43+0000


## Paths

In [3]:
DB_PATH   = r"ai_sandbox_PSA_16_Oct_2025_MLABite.db"
CSV_PATH  = r"MLABiTe/20250704_160557/en_us/OpenAIGPT35Turbo/20250704160636_global_evaluation.csv"

In [4]:
df_mlabite = pd.read_csv(CSV_PATH, sep=';')

## Some delcarations

In [53]:
PROJECT_NAME   = "ai-sandbox"
PROJECT_STATUS = "Processing"

TOOL_NAME    = "GlobalEval Loader"
TOOL_SOURCE  = "CSV Import"
TOOL_VERSION = "1.0"
TOOL_LICENSE = "Open-source"

CONFIG_NAME        = "GlobalEval Config"
CONFIG_DESCRIPTION = "Config for global evaluation import"

EVAL_STATUS   = "Processing"
DATASET_TYPE  = "Validation"      # adjust to your enum
DATASET_SRC   = "Global Evaluation CSV"
DATASET_VER   = "v1"
DATASET_LIC   = "Open-source"     # adjust to your enum

DEFAULT_UNIT = ""
NON_METRIC_COLUMNS = {"language", "split", "subset", "run_id"}

## Helpers

In [54]:
def table_cols(conn, table):
    try:
        return {r[1] for r in conn.execute(f"PRAGMA table_info('{table}');").fetchall()}
    except sqlite3.OperationalError:
        return set()

def one(conn, q, params=()):
    cur = conn.execute(q, params); row = cur.fetchone(); cur.close()
    return row[0] if row else None

def exists_table(conn, name):
    return one(conn, "SELECT 1 FROM sqlite_master WHERE type='table' AND name=?", (name,)) is not None

def insert_dynamic(conn, table, data):
    cols = table_cols(conn, table)
    payload = {k: v for k, v in data.items() if k in cols}
    keys = ", ".join(payload.keys())
    qms  = ", ".join(["?"]*len(payload))
    if not keys:
        raise RuntimeError(f"No matching columns to insert into {table}")
    cur = conn.execute(f"INSERT INTO {table} ({keys}) VALUES ({qms})", tuple(payload.values()))
    return cur.lastrowid

def update_dynamic(conn, table, pk_field, pk_value, updates):
    cols = table_cols(conn, table)
    payload = {k: v for k, v in updates.items() if k in cols}
    if not payload:
        return
    sets = ", ".join([f"{k}=?" for k in payload.keys()])
    conn.execute(f"UPDATE {table} SET {sets} WHERE {pk_field}=?", (*payload.values(), pk_value))

# If you have joined inheritance and NOT NULL element.project_id:
def ensure_element(conn, type_spec, name, description, project_id):
    if not exists_table(conn, "element"):
        return None
    eid = one(conn, "SELECT id FROM element WHERE type_spec=? AND name=? AND project_id=?",
              (type_spec, name, project_id))
    if eid:
        update_dynamic(conn, "element", "id", eid, {"description": description})
        return eid
    return insert_dynamic(conn, "element", {
        "type_spec": type_spec, "name": name, "description": description, "project_id": project_id
    })


In [55]:
# --- Upserts (with required FKs) ---
def ensure_project(conn, name, status):
    if not exists_table(conn, "project"):
        raise RuntimeError("Table 'project' not found.")
    pid = one(conn, "SELECT id FROM project WHERE name=?", (name,))
    if pid:
        update_dynamic(conn, "project", "id", pid, {"status": status})
        return pid
    return insert_dynamic(conn, "project", {"name": name, "status": status})

def ensure_configuration(conn, name, description):
    if not exists_table(conn, "configuration"):
        raise RuntimeError("Table 'configuration' not found.")
    cid = one(conn, "SELECT id FROM configuration WHERE name=?", (name,))
    if cid:
        update_dynamic(conn, "configuration", "id", cid, {"description": description})
        return cid
    return insert_dynamic(conn, "configuration", {"name": name, "description": description})

def ensure_tool(conn, name, source, version, licensing, project_id):
    if not exists_table(conn, "tool"):
        return None
    tid = one(conn, "SELECT id FROM tool WHERE name=? AND version=?", (name, version))
    if tid:
        update_dynamic(conn, "tool", "id", tid, {"source": source, "licensing": licensing})
        return tid
    base_id = ensure_element(conn, "Tool", name, "", project_id)
    return insert_dynamic(conn, "tool", {
        "id": base_id, "name": name, "source": source, "version": version, "licensing": licensing
    })

def ensure_evaluation(conn, status, project_id, config_id):
    # IMPORTANT: this table requires project_id & config_id (NOT NULL)
    if not exists_table(conn, "evaluation"):
        return None
    eid = one(conn, "SELECT id FROM evaluation WHERE project_id=? AND config_id=? LIMIT 1",
              (project_id, config_id))
    if eid:
        update_dynamic(conn, "evaluation", "id", eid, {"status": status})
        return eid
    return insert_dynamic(conn, "evaluation", {
        "status": status, "project_id": project_id, "config_id": config_id
    })

def ensure_datashape(conn, accepted_target_values, project_id):
    if not exists_table(conn, "datashape"):
        return None
    atv_json = json.dumps(accepted_target_values, ensure_ascii=False)
    did = one(conn, "SELECT id FROM datashape WHERE accepted_target_values=?", (atv_json,))
    if did:
        return did
    base_id = ensure_element(conn, "Datashape", "global_eval_shape", "Accepted structure for global eval", project_id)
    return insert_dynamic(conn, "datashape", {"id": base_id, "accepted_target_values": atv_json})

def ensure_dataset(conn, name, datashape_id, project_id, source, version, licensing, dtype):
    if not exists_table(conn, "dataset"):
        return None
    dsid = one(conn, "SELECT id FROM dataset WHERE source=? AND version=? AND type=? AND datashape_id=? LIMIT 1",
               (source, version, dtype, datashape_id))
    if dsid:
        return dsid
    base_id = ensure_element(conn, "Dataset", name, "", project_id)
    return insert_dynamic(conn, "dataset", {
        "id": base_id, "source": source, "version": version,
        "licensing": licensing, "type": dtype, "datashape_id": datashape_id
    })

def ensure_observation(conn, name, observer, whenObserved, tool_id, evaluation_id, dataset_id, project_id):
    # REQUIRED columns in your DB: whenObserved, tool_id, eval_id, dataset_2_id (all NOT NULL)
    if not exists_table(conn, "observation"):
        return None
    oid = one(conn, "SELECT id FROM observation WHERE name=?", (name,))
    payload = {
        "observer": observer,
        "whenObserved": whenObserved,
        "tool_id": tool_id,
        "eval_id": evaluation_id,
        "dataset_2_id": dataset_id
    }
    if oid:
        update_dynamic(conn, "observation", "id", oid, payload)
        return oid
    base_id = ensure_element(conn, "Observation", name, "", project_id)
    payload.update({"id": base_id, "name": name, "description": f"Imported {name}"})
    return insert_dynamic(conn, "observation", payload)

def ensure_metric(conn, metric_name, description, project_id):
    if not exists_table(conn, "metric"):
        return None
    mid = one(conn, "SELECT id FROM metric WHERE name=?", (metric_name,))
    if mid:
        return mid
    base_id = ensure_element(conn, "Metric", metric_name, description, project_id)
    return insert_dynamic(conn, "metric", {"id": base_id, "name": metric_name, "description": description})

def insert_measure(conn, metric_id, observation_id, measurand_id, value, unit=DEFAULT_UNIT, uncertainty=0.0, error=""):
    if not exists_table(conn, "measure"):
        return None
    return insert_dynamic(conn, "measure", {
        "metric_id": metric_id,
        "observation_id": observation_id,
        "measurand_id": measurand_id,  # <- REQUIRED in your schema
        "value": str(value),
        "unit": unit,
        "uncertainty": float(uncertainty),
        "error": error
    })


## Populate DB

In [56]:
def main():
    # 1) Load CSV
    df = pd.read_csv(CSV_PATH, sep=';')

    # 2) Group by slice (language) if present
    if "language" in df.columns:
        groups = df.groupby("language")
    else:
        df["_slice"] = "global"
        groups = df.groupby("_slice")

    conn = sqlite3.connect(DB_PATH)
    conn.execute("PRAGMA foreign_keys = ON;")

    with conn:
        # 3) Required parents FIRST
        project_id = ensure_project(conn, PROJECT_NAME, PROJECT_STATUS)
        config_id  = ensure_configuration(conn, CONFIG_NAME, CONFIG_DESCRIPTION)
        tool_id    = ensure_tool(conn, TOOL_NAME, TOOL_SOURCE, TOOL_VERSION, TOOL_LICENSE, project_id)
        eval_id    = ensure_evaluation(conn, EVAL_STATUS, project_id, config_id)
        datashape_id = ensure_datashape(conn, {"schema": "wide_metrics"}, project_id)

        # 4) Per-slice dataset + observation, then measures
        inserted = 0
        for slice_key, sub in groups:
            ds_name = f"global_eval__{slice_key}"
            ds_id   = ensure_dataset(conn, ds_name, datashape_id, project_id,
                                     DATASET_SRC, DATASET_VER, DATASET_LIC, DATASET_TYPE)

            when_ts = when_now#datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")
            obs_name = f"obs__{slice_key}__{Path(CSV_PATH).stem}"
            obs_id   = ensure_observation(conn, obs_name, TOOL_NAME, when_ts,
                                          tool_id, eval_id, ds_id, project_id)

            candidate_cols = [c for c in sub.columns if c not in NON_METRIC_COLUMNS]
            num_df = sub[candidate_cols].select_dtypes(include=["number"])
            if num_df.empty:
                continue

            row = num_df.mean(numeric_only=True)   # aggregate to one measure per metric per slice
            for col, val in row.items():
                metric_name = col
                mid = ensure_metric(conn, metric_name, f"Imported from {Path(CSV_PATH).name}", project_id)
                insert_measure(conn, mid, obs_id, ds_id, float(val), unit=DEFAULT_UNIT)
                inserted += 1

        print(f"Done. Slices: {len(list(groups))}, measures inserted: {inserted}")


## Main

In [57]:
if __name__ == "__main__":
    main()

IntegrityError: NOT NULL constraint failed: metric.type_spec